In [4]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [5]:
C=70.0
downweight=0.8
risk_spill=0.95
risk_spoil=risk_spill
downweight_granularity=['dtime','Direction','dday','month']

In [6]:
C_datadriven=pd.read_csv('R_Training_Pax.csv')
C_datadriven=C_datadriven.groupby(['DepDate']+downweight_granularity)['NumPax'].sum().reset_index()
C_datadriven=C_datadriven.groupby(downweight_granularity)['NumPax'].max().reset_index().rename(columns={'NumPax': 'downweight'})
C_datadriven['downweight']=C_datadriven['downweight'].astype('float')/C
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: downweight if x>downweight else x)
# C_datadriven['downweight']=downweight

In [7]:
Prediction=pd.read_csv('R_Output_Training_Pax.csv',sep=',')

Tree=Prediction[['yday','dtime','Direction','month','dday','forecast_node']].drop_duplicates().copy()

Tree_yday=Tree.groupby('forecast_node')['yday'].min().reset_index().merge(Tree.groupby('forecast_node')['yday'].max().reset_index(),
on='forecast_node').rename(columns={'yday_x': 'Min', 'yday_y': 'Max'})

Tree_yday['yday']='['+Tree_yday['Min'].astype('str')+'-'+Tree_yday['Max'].astype('str')+']'
Tree_yday=Tree_yday[[x for x in Tree_yday.columns if x not in ['Min','Max']]]

Tree_dtime=Tree.groupby('forecast_node')['dtime'].unique().reset_index()
Tree_dday=Tree.groupby('forecast_node')['dday'].unique().reset_index()
Tree_Direction=Tree.groupby('forecast_node')['Direction'].unique().reset_index()

Tree=Tree_yday.merge(Tree_dtime,on='forecast_node')
Tree=Tree.merge(Tree_dday,on='forecast_node')
Tree=Tree.merge(Tree_Direction,on='forecast_node')

In [8]:
Prediction=Prediction.merge(C_datadriven,on=downweight_granularity)

IdealCurve=Prediction[['Dprio']+downweight_granularity+['forecast_bookings','forecast_node','downweight']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)

IdealCurve=IdealCurve.merge(IdealCurve.groupby(downweight_granularity)['forecast_bookings'].sum().reset_index()\
.rename(columns={'forecast_bookings': 'forecast_bookings_sum'}),on=downweight_granularity)

In [9]:
IdealCurve['forecast_bookings_full']=C*(IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum'])
IdealCurve['forecast_bookings_downweighted']=C*IdealCurve['downweight']*(IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum'])

In [10]:
IdealCurve['Cap_full']=C
IdealCurve['Cap_downweighted']=C*IdealCurve['downweight']

In [11]:
Probability=IdealCurve[['Dprio','dtime','Direction','dday','month','Cap_full','Cap_downweighted']].copy()

In [12]:
IdealCurve.to_csv('IdealCurve_forYield.csv',index=False)

# Cumulation

In [13]:
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if 'sum' not in x and x!='forecast_bookings']]

In [14]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity+['downweight'])

IdealCurve_average_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_full'].cumsum().reset_index().rename(columns={'forecast_bookings_full': 'Ideal_average_full'})
IdealCurve_average_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_downweighted'].cumsum().reset_index().rename(columns={'forecast_bookings_downweighted': 'Ideal_average_downweighted'})

IdealCurve_average_full['Dprio']=-IdealCurve_average_full['Dprio']
IdealCurve_average_downweighted['Dprio']=-IdealCurve_average_downweighted['Dprio']

In [15]:
IdealCurve=IdealCurve.reset_index()
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity+['downweight'])
IdealCurve_intermediate_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_full'].cumsum().reset_index().rename(columns={'forecast_bookings_full': 'Ideal_intermediate_full'})
IdealCurve_intermediate_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_downweighted'].cumsum().reset_index().rename(columns={'forecast_bookings_downweighted': 'Ideal_intermediate_downweighted'})

In [16]:
Probability=Probability.merge(IdealCurve_intermediate_full[[x for x in IdealCurve_intermediate_full if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_intermediate_downweighted[[x for x in IdealCurve_intermediate_downweighted if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_average_full[[x for x in IdealCurve_average_full if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_average_downweighted[[x for x in IdealCurve_average_downweighted if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

In [17]:
Probability.to_csv('Probability.csv',index=False)

In [18]:
IdealCurve=IdealCurve_average_full.merge(IdealCurve_average_downweighted,on=['Dprio']+downweight_granularity+['downweight'])
IdealCurve=IdealCurve.merge(IdealCurve_intermediate_full,on=['Dprio']+downweight_granularity+['downweight'])
IdealCurve=IdealCurve.merge(IdealCurve_intermediate_downweighted,on=['Dprio']+downweight_granularity+['downweight'])

In [19]:
IdealCurve['Ideal_lower']=C*IdealCurve['downweight']-poisson.ppf(risk_spoil,IdealCurve['Ideal_intermediate_downweighted'])
IdealCurve['Ideal_lower']=IdealCurve['Ideal_lower'].apply(lambda x: 0 if x<0 else x)

IdealCurve['Ideal_upper']=C-poisson.ppf(1-risk_spill,IdealCurve['Ideal_intermediate_full'])
IdealCurve['Ideal_upper']=IdealCurve['Ideal_upper'].apply(lambda x: C if x>C else x)

In [20]:
IdealCurve=IdealCurve[downweight_granularity+['Dprio','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]
IdealCurve.to_csv('IdealCurve.csv',index=False)

In [21]:
# pick_node=3

# fig=plt.figure(1,figsize=(15,8))

# plt.plot(IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Dprio'],
# IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Ideal_average_full'],
# color='green',linestyle='--',label='Normal booking curve (LF = 100%)')
# plt.plot(IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Dprio'],
# IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Ideal_average_downweighted'],color='blue',
#         linestyle='--',label='Normal booking curve (LF = '+str(downweight*100)+'%)')
# plt.plot(IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Dprio'],
# IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Ideal_lower'],color='orange',
#         linestyle='--',label='Steering frontier')
# plt.plot(IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Dprio'],
# IdealCurve.loc[IdealCurve['forecast_node']==pick_node,'Ideal_upper'],color='orange')

# # plt.fill_between(IdealCurve['forecast_node']==pick_node,'Dprio'],0,
# # np.array(bookcurve_lowbound)[range(len(poisson_lambda)-1,-1,-1)],color='orange',alpha=0.25)
# # plt.fill_between(range(len(poisson_lambda)),C,np.array(bookcurve_upbound)[range(len(poisson_lambda)-1,-1,-1)],color='orange',alpha=0.25)


# plt.axhline(0,linewidth=2,color='black',linestyle=':')
# plt.axhline(C,linewidth=2,color='black',linestyle=':')
    
# plt.xlabel('Days till departure',size=15)
# plt.ylabel('Bookings',size=15)
# plt.xlim(0,366)

# plt.title('Illustration of booking curve steering mechanism',size=20)

# plt.legend(prop={'size': 12},bbox_to_anchor=(1.01, 0.95), loc=2, borderaxespad=0.)

# # plt.text(365.0/2,np.mean([bookcurve_upbound[250],C]),'Phase-Down Area',fontsize=24,color='orange')
# # plt.text(10,np.float(bookcurve_upbound[250])/2,'Ramp-Up Area',fontsize=24,color='orange')

# plt.annotate('Capacity',xy=(30,C-1),xytext=(50,C-20),arrowprops=dict(facecolor='black',edgecolor='black',shrink=0.05),
#              color='black',fontsize='20')

# plt.show()